## Prepare the workspace

In [1]:
# Check torch version and CUDA status if GPU is enabled.
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled. 

2.0.1
False


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [2]:
# Imports here
import time
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import time
from collections import OrderedDict
import os
import math
import numpy as np
from PIL import Image
import seaborn as sb

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). 

If you do not find the `flowers/` dataset in the current directory, **/workspace/home/aipnd-project/**, you can download it using the following commands. 

**Method 1**
```bash
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers
```


**Method 2**
```bash
!cp -r /data/ .
```

## Data Description
The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [3]:
data_dir = 'flowers'
_dirs = {
    'train': data_dir + '/train',
    'valid': data_dir + '/valid',
    'test' : data_dir + '/test'}

In [4]:
_hp = _hyperparms = {
    'batch_size': 32, 
    'epochs'    : 7, # was 10, now 0 since we just reload checkpoints
    'lr'        : 0.003,
    'criterion' : nn.NLLLoss()
}
print("hyper parms: " + str(_hp))

# global variables
_gv =  {
    'device'           : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'print_every'      : 25,
    'checkpoint_every' : 25
}
print("global vars: " +str(_gv))

hyper parms: {'batch_size': 32, 'epochs': 7, 'lr': 0.003, 'criterion': NLLLoss()}
global vars: {'device': device(type='cpu'), 'print_every': 25, 'checkpoint_every': 25}


In [5]:
# from "part 8 transfer learning"

# augment training data with  rot, flip
_transforms = {
    'train': transforms.Compose([transforms.RandomRotation(30),
                                  transforms.RandomResizedCrop(224),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],
                                                       [0.229, 0.224, 0.225])]),
    # only resize, crop, normalize for validation and test
    'valid': transforms.Compose([transforms.Resize(255),
                                  transforms.CenterCrop(224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],
                                                       [0.229, 0.224, 0.225])])}
_transforms['test'] = _transforms['valid']

In [6]:
# adaptedfrom "part 8 transfer learning"

# Load the datasets with ImageFolder
_sets = {
    'train': datasets.ImageFolder(_dirs['train'], transform=_transforms['train']),
    'test': datasets.ImageFolder(_dirs['test'], transform=_transforms['test']),
    'valid': datasets.ImageFolder(_dirs['valid'], transform=_transforms['valid'])
}

# Using the image datasets and the trainforms, define the dataloaders
_loaders = {
    'train' :  torch.utils.data.DataLoader(_sets['train'], batch_size=_hp['batch_size'], shuffle=True),
    # don't shuffle for test and validate
    'test'  :  torch.utils.data.DataLoader(_sets['test'], batch_size=_hp['batch_size']),
    'valid' :torch.utils.data.DataLoader(_sets['valid'], batch_size=_hp['batch_size'])}



In [7]:
for ldr in ['train', 'test', 'valid']:
    print(f"dataset len({ldr}) = " + str(len(_sets[ldr])))
for ldr in ['train', 'test', 'valid']:
    print(f"dataldr len({ldr}) = " + str(len(_loaders[ldr])))
print(f"steps/epoch = {math.ceil(len(_loaders['train']) / _hp['batch_size'])}")

dataset len(train) = 6552
dataset len(test) = 819
dataset len(valid) = 818
dataldr len(train) = 205
dataldr len(test) = 26
dataldr len(valid) = 26
steps/epoch = 7


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [8]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. 

## Note for Workspace users: 
If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [9]:
# Define the loss

def get_classifier():
    dict = OrderedDict([('fc1', nn.Linear(25088, 4096)),
                        ('relu', nn.ReLU()), 
                        ('fc2', nn.Linear(4096, 102)),
                        ('output', nn.LogSoftmax(dim=1))])
    classifier = nn.Sequential(dict)
    return classifier

images, labels = next(iter(_loaders['train'])) # get  batch

model = models.vgg16(pretrained=True)
model.class_to_idx = _sets['train'].class_to_idx
model.arch = 'vgg16'
print(model)

for param in model.parameters(): #freeze parms
    param.requires_grad = False
    
model.classifier = nn.Sequential(get_classifier())

# feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=_hp['lr'])
model.to(_gv['device']);

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
# Save the checkpoint (adapted from lesson 6.5)
# move these to an earlier cell so that we can
# periodically check DURING training so we can
# stop wasting GPU time if it times out on us.
#
# with this we can restart and only lose the steps
# since the last checkpoint

chkpt_path = "checkpoint.pth"

def save_chkpt(model, optimizer, epoch, step):
    checkpoint = {'arch'        : model.arch,
                  'epoch'       : epoch,
                  'step'        : step,
                  'features'    : model.features,
                  'class_to_idx': model.class_to_idx,
                  'classifier'  : model.classifier,
                  'optimizer'   : optimizer.state_dict(),
                  'state_dict'  : model.state_dict(),
                  'losslog'     : model.losslog
                 }
    torch.save(checkpoint, chkpt_path)

# function to load into an already constructed model
def load_chkpt(model):
    if torch.cuda.device_count():
        checkpoint = torch.load(chkpt_path)
    else: #map_location=torch.device("cpu"
        checkpoint = torch.load(chkpt_path, map_location=torch.device("cpu"))
        
    model.arch         = checkpoint['arch'] 
    model.epoch        = checkpoint['epoch'] 
    model.step         = checkpoint['step'] 
    model.features     = checkpoint ['features']
    model.class_to_idx = checkpoint ['class_to_idx']
    model.classifier   = checkpoint ['classifier']
    model.optimizer    = checkpoint ['optimizer']
    model.losslog      = checkpoint['losslog']
    model.load_state_dict(checkpoint ['state_dict'])
    # no need to refreeze pretrained parameters
    
    return None # in case the unwary expect this to create and return the model

In [11]:
# utility stuff
def yes_or_no(question):
    while True:
        answer = input(f"{question} (yes/no): ").lower().strip()
        if answer in ['yes', 'y']:
            return True
        elif answer in ['no', 'n']:
            return False
        else:
            print("Please enter 'yes' or 'no'.")

def show_elapsed_mins(t0):
    elapsed_min = (time.time() - t0) / 60
    print(f"\t elapsed(min) = {elapsed_min:.2f}")

In [12]:
def do_train():
    steps = 0
    running_loss = 0
    chkpt_every = 10
    t0 = time.time()
    
    for epoch in range(model.epoch, _hp['epochs']): 
        print("epoch: " + str(epoch))

        for inputs, labels in _loaders['train']:
            steps += 1

            # Move input and label tensors to the default device
            inputs, labels = inputs.to(_gv['device']), labels.to(_gv['device'])    
            import pdb;pdb.set_trace()
            optimizer.zero_grad()
            logps = model.forward(inputs)
            loss = _hp['criterion'](logps, labels)
            loss.backward()
            optimizer.step()
        
            if steps % _gv['checkpoint_every'] == 0:
                show_elapsed_mins(t0)
                print(f"\t saving checkpt at step {steps}")
                save_chkpt(model, optimizer, epoch, steps)            

            running_loss += loss.item()

            if steps % _gv['print_every'] == 0:
                print(f"step {steps}")
                show_elapsed_mins(t0)
                test_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in _loaders['valid']:
                        inputs, labels = inputs.to(_gv['device']), labels.to(_gv['device'])
                        logps = model.forward(inputs)
                        batch_loss = _hp['criterion'](logps, labels)
                    
                        test_loss += batch_loss.item()
                    
                        # Calculate accuracy
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
             
                stats = {'epoch' : f"{epoch+1}/{_hp['epochs']}",
                         'train_loss'    : f"{running_loss/_gv['print_every']:.3f}",
                         'test_loss'     : f"{test_loss/len(_loaders['test']):.3f} ",
                         'test_accuracy' : f"{accuracy/len(_loaders['test']):.3f}"}

                print(stats)
                model.losslog.append(stats)
                running_loss = 0
                model.train()

In [ ]:
if os.path.isfile(chkpt_path):
    print("checkpoint exists")
    if yes_or_no("Load existing checkpoint"):
        load_chkpt(model)
        steps = model.step
        print(f"last checkpoint step: {steps}")
    if yes_or_no("train"):
        print("training beyond checkpoint")
        do_train()
    else:
        print("not extending training")
else:
    print("checkpoint not found, train")
    do_train()

checkpoint exists
Load existing checkpoint (yes/no): y
last checkpoint step: 1750
train (yes/no): y
training beyond checkpoint
epoch: 6
> /tmp/ipykernel_76/2657453286.py(16)do_train()
     14             inputs, labels = inputs.to(_gv['device']), labels.to(_gv['device'])
     15             import pdb;pdb.set_trace()
---> 16             optimizer.zero_grad()
     17             logps = model.forward(inputs)
     18             loss = _hp['criterion'](logps, labels)

ipdb> l
     11             steps += 1
     12 
     13             # Move input and label tensors to the default device
     14             inputs, labels = inputs.to(_gv['device']), labels.to(_gv['device'])
     15             import pdb;pdb.set_trace()
---> 16             optimizer.zero_grad()
     17             logps = model.forward(inputs)
     18             loss = _hp['criterion'](logps, labels)
     19             loss.backward()
     20             optimizer.step()
     21 

ipdb> n
> /tmp/ipykernel_76/2657453286.

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
def test_net():
    print("test it")
    correct = 0
    total = 0
    n = 0
    with torch.no_grad():
        for data in _loaders['test']:
            print(f"\t n = {n}");n += 1
            images, labels = data[0].to(_gv['device']), data[1].to(_gv['device'])
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    img_cnt = len(_sets['test'])
    print(f"Accuracy of the network on the {img_cnt} test images {accuracy}%")

if yes_or_no("test accuracy?"): # skip this to save time if value is known from prev runs
    test_net() 


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# : Write a function that loads a checkpoint and rebuilds the model(already used above)
# load_chkpt(model)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def get_scaled_image(image_path):
    im = Image.open(image_path)
    # https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.size
    dims = (list(im.size)) #[width, height] 
    # https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.thumbnail
    short_side = dims.index(min(dims))
    dims[short_side] = 256 # shortest side now 256
    MAXINT_32 = 2**31 - 1
    if short_side == 0: # width
        im.thumbnail((dims[short_side],MAXINT_32), Image.ANTIALIAS)
    else:
        im.thumbnail((MAXINT_32, dims[short_side]), Image.ANTIALIAS)
        
    w, h = dims = im.size #im resize in place, get new dims, tuple because dont need to alter
    model_img_size = 224

    top = (h - model_img_size)/2
    left = (w - model_img_size)/2
    im = im.crop((left, top, left + model_img_size, top + model_img_size))
    return im
                          
def gen_np_img(pil_image, mean, stdev):
    np_image = np.array(pil_image) / 255
    # per instructions above You'll want to subtract the means from each color channel, then divide by the 
    # standard deviation.
    np_image -= np.array(mean) 
    np_image /= np.array(stdev)
    np_image= np_image.transpose((2,0,1))
    return np_image
                          
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    im = get_scaled_image(image_path)
    np_im = gen_np_img(im, [0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    return np_im

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = np.transpose(image, (1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topkl):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path) #loading image and processing it using above defined function
    im = torch.from_numpy(image).type(torch.FloatTensor) # np array -> tensor
    im = im.unsqueeze(dim = 0) # batch size == 1 for inference 
        
    with torch.no_grad():
        output = model.forward(im)
        
    logps = torch.exp(output)
    probs, indices = logps.topk(topkl)
    probs = probs.numpy() # tensor -> nparray
    indexes = indices.numpy() 
    
    prob_list = probs.tolist()[0]
    ix_list = indices.tolist()[0]
    mapping = {val: key for key, val in model.class_to_idx.items()}
    
    classes = [mapping [item] for item in ix_list]
    classes = np.array(classes)
    
    return probs, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
img_fname = 'flowers/train/1/image_06734.jpg'
image = process_image(img_fname)
imshow(image)

probs, classes = predict(img_fname, model, 5)
class_names = [cat_to_name [item] for item in classes]
plt.figure(figsize = (6,10))
plt.subplot(2,1,2)
probs = np.reshape(probs, -1)
sb.barplot(x=probs, y=class_names, color= 'red');
plt.show()


## Reminder for Workspace users
If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.

In [ ]:
# remove .pth files or move it to a temporary `~/opt` directory in this Workspace

if yes_or_no("remove the checkpoint?"):
    !rm checkpoint.pth